In [16]:
import cv2
import mediapipe as mp
import numpy as np
import math
import csv
import os
from datetime import datetime

# Инициализация Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1)

# Конфигурация
CSV_FILENAME = "face_analysis_results.csv"
FOREHEAD_POINTS = [103, 332, 333, 298]
BROW_POINTS = {'left': [70, 63, 105, 66, 107], 'right': [336, 296, 334, 293, 300]}
HEAD_CONTOUR = [10, 338, 297, 332, 284, 251]

def calculate_distance(p1, p2, image_shape):
    h, w = image_shape[:2]
    return math.sqrt((p1.x*w - p2.x*w)**2 + (p1.y*h - p2.y*h)**2)

def save_to_csv(data, filename):
    file_exists = os.path.isfile(filename)
    
    with open(filename, 'a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=data.keys())
        
        if not file_exists:
            writer.writeheader()
        
        writer.writerow(data)

def analyze_and_save(image_path):
    # Инициализация данных
    analysis_data = {
        'filename': os.path.basename(image_path),
        'analysis_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'forehead_shape': None,
        'brow_position': None,
        'head_shape': None,
        'forehead_ratio': None,
        'brow_height_avg': None,
        'head_ratio': None
    }

    try:
        image = cv2.imread(image_path)
        if image is None:
            raise FileNotFoundError(f"Файл {image_path} не найден")

        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        if not results.multi_face_landmarks:
            print("Лицо не обнаружено")
            return

        landmarks = results.multi_face_landmarks[0].landmark
        h, w = image.shape[:2]

        # Анализ лба
        forehead_width_top = calculate_distance(landmarks[103], landmarks[332], (h,w))
        forehead_width_bottom = calculate_distance(landmarks[333], landmarks[298], (h,w))
        forehead_ratio = forehead_width_top / forehead_width_bottom
        analysis_data['forehead_ratio'] = round(forehead_ratio, 3)
        analysis_data['forehead_shape'] = "Суженный" if forehead_ratio < 0.95 else "Прямоугольный"

        # Анализ бровей
        brow_heights = []
        for side in ['left', 'right']:
            brow_pts = [landmarks[i] for i in BROW_POINTS[side]]
            eye_pts = [landmarks[145], landmarks[159]] if side == 'left' else [landmarks[374], landmarks[386]]
            
            brow_center = np.mean([(p.y*h) for p in brow_pts])
            eye_center = np.mean([(p.y*h) for p in eye_pts])
            brow_heights.append(eye_center - brow_center)
        
        avg_brow_height = np.mean(brow_heights)
        analysis_data['brow_height_avg'] = round(avg_brow_height, 1)
        analysis_data['brow_position'] = "Высокие" if avg_brow_height > 15 else "Низкие"

        # Анализ формы головы
        head_points = [landmarks[i] for i in HEAD_CONTOUR]
        head_width = calculate_distance(head_points[0], head_points[3], (h,w))
        head_height = calculate_distance(head_points[1], head_points[4], (h,w))
        head_ratio = head_width / head_height
        analysis_data['head_ratio'] = round(head_ratio, 2)
        analysis_data['head_shape'] = "Вертикальный" if head_ratio < 0.65 else "Горизонтальный"

        # Сохранение в CSV
        save_to_csv(analysis_data, CSV_FILENAME)
        print(f"Данные сохранены в {CSV_FILENAME}")

        # Визуализация (опционально)
        for idx in FOREHEAD_POINTS + BROW_POINTS['left'] + BROW_POINTS['right'] + HEAD_CONTOUR:
            x = int(landmarks[idx].x * w)
            y = int(landmarks[idx].y * h)
            cv2.circle(image, (x,y), 2, (0,255,0), -1)

        cv2.imshow("Analysis Result", image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    except Exception as e:
        print(f"Ошибка: {str(e)}")

# Пример использования
analyze_and_save("../data/21.jpg")

Данные сохранены в face_analysis_results.csv


KeyboardInterrupt: 